In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import pickle
from matplotlib import pylab as plt
%matplotlib inline
from IPython.display import *

In [37]:
df_demo = pickle.load(open('/Users/boris/temp/df_demo.pkl', 'rb'))

In [53]:
def regression_data(df_demo):
    true_true = df_demo.loc[df_demo.true_true][['age', 'wt', 'sex']].dropna()
    true_true['side_effect'] = True
    true_true['exposure'] = 1

    true_false = df_demo.loc[df_demo.true_false][['age', 'wt', 'sex']].dropna()
    true_false['side_effect'] = False
    true_false['exposure'] = 1

    false_true = df_demo.loc[df_demo.drug_naive_true][['age', 'wt', 'sex']].dropna()
    false_true['exposure'] = 0
    false_true['side_effect'] = 1

    false_false = df_demo.loc[df_demo.drug_naive_false][['age', 'wt', 'sex']].dropna()
    false_false['exposure'] = 0
    false_false['side_effect'] = 0


    df_regression = pd.concat([true_true, true_false, false_true, false_false], sort=False).reset_index(drop=True)
    df_regression['is_female'] = df_regression['sex'] == 'F'
    df_regression.drop('sex', axis=1, inplace=True)
    df_regression['intercept'] = 1.0
    regression_cols = [c for c in df_regression.columns if c != 'side_effect']
    df_regression.side_effect = df_regression.side_effect.astype(int)
    df_regression.is_female = df_regression.is_female.astype(int)
    return df_regression, regression_cols, 'side_effect'
    
def regression(df_demo, name):
    df_regression, regression_cols, column_y = regression_data(df_demo)

    logit = sm.Logit(df_regression[column_y], df_regression[regression_cols])
    result = logit.fit()
    html_summary = '<h1>' + name + '</h1>\n' +\
    result.summary2(title=name).as_html() \
        + '\n<br>\n' \
    + colinearity_analysis(df_regression=df_regression, regression_cols=regression_cols, name=None)

    return html_summary



In [54]:
df_regression, regression_cols, column_y = regression_data(df_demo)
def colinearity_analysis(df_regression, regression_cols, name=None):
    rows = []
    if name:
        row = f'{name}: '
    else:
        row = ''
    rows.append('<b> ' + row + 'variance inflation factors' + '</b>')
    rows.append('<table><tbody>')
    rows.append('''<tr>
			<th>variable</th>
			<th>VIF</th>
		</tr>
    ''')
    
    mat = df_regression[regression_cols].values
    for i in range(len(regression_cols)):
        colname = regression_cols[i]
        if colname == 'intercept': 
            continue
        vif = variance_inflation_factor(mat, i)
        rows.append(f'<tr><td>{colname:30s}</td><td>{vif:.3f}</td></tr>')
    rows.append('</tbody></table>')
    return '\n'.join(rows)

In [55]:
html = regression(df_demo, 'x')
HTML(html)

Optimization terminated successfully.
         Current function value: 0.155700
         Iterations 8


In [52]:
HTML(html)

In [26]:
HTML(colinearity_analysis(df_regression, regression_cols))

variable,VIF
age,1.000
wt,1.024
exposure,1.001
is_female,1.024


In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [14]:
mat = df_regression[regression_cols].values
for i in range(len(regression_cols)):
    colname = regression_cols[i]
    if colname == 'intercept': 
        continue
    vif = variance_inflation_factor(mat, i)
    print

age                            1.000
wt                             1.024
exposure                       1.001
is_female                      1.024


In [13]:
N = 100
x = np.arange(N)
df = pd.DataFrame({'x0': x })
for i in range(1, 10):
    df[f'x{i}'] = df[f'x{i-1}'] + np.random.randn(N) * 10
df['x99'] = np.random.randn(N)
mat = df.values
for i in range(df.shape[1]):
    vif = variance_inflation_factor(mat, i)
    print(f'{i:5d} {vif:.3f}')

    0 36.130
    1 82.763
    2 75.398
    3 95.318
    4 84.452
    5 95.172
    6 100.898
    7 96.693
    8 73.240
    9 39.070
   10 1.220


In [35]:
HTML(result.summary(title='abclkj lkj ').as_html())

In [39]:
summary = result.summary2()
summary

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.012      
Dependent Variable: side_effect      AIC:              287351.9078
Date:               2018-07-03 00:11 BIC:              287410.5833
No. Observations:   922745           Log-Likelihood:   -1.4367e+05
Df Model:           4                LL-Null:          -1.4537e+05
Df Residuals:       922740           LLR p-value:      0.0000     
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     8.0000                                        
-------------------------------------------------------------------
              Coef.   Std.Err.      z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
age           0.0000    0.0000     3.8854  0.0001   0.0000   0.0000
wt            0.0011    0.0002     7.0872  0.0000   0.0008   0.0014
exposure     -0.0032    0.1654    -0.0193  0.9846  -0.3274   0.3210
is_female    -0.6258    0.0112   -55.6263  0.0000  -0.6478  -0.6037
intercept    -3.0207    0.0150  -201.1556  0.0000  -3.0502  -2.9913
==================================================================

"""

In [40]:
summary.tables

[                     0                 1                  2            3
 0               Model:             Logit  Pseudo R-squared:        0.012
 1  Dependent Variable:       side_effect               AIC:  287351.9078
 2                Date:  2018-07-03 00:11               BIC:  287410.5833
 3    No. Observations:            922745    Log-Likelihood:  -1.4367e+05
 4            Df Model:                 4           LL-Null:  -1.4537e+05
 5        Df Residuals:            922740       LLR p-value:       0.0000
 6           Converged:            1.0000             Scale:       1.0000
 7      No. Iterations:            8.0000                                ,
               Coef.      Std.Err.           z         P>|z|    [0.025  \
 age        0.000003  7.560923e-07    3.885388  1.021666e-04  0.000001   
 wt         0.001117  1.576570e-04    7.087219  1.368338e-12  0.000808   
 exposure  -0.003195  1.654145e-01   -0.019315  9.845895e-01 -0.327402   
 is_female -0.625785  1.124980e-02  -

In [27]:
np.exp(result.params)

age          1.055009
wt           1.004223
is_female    0.301520
intercept    0.001923
dtype: float64

In [6]:
df_demo.shape

(5385720, 9)

In [9]:
df_demo.loc[df_demo.caseid == 9319489]

,caseid,true_true,true_false,drug_naive_true,drug_naive_false,age,sex,wt,event_date
308889,9319489,False,False,False,True,56.763859,F,NaN,2015-08-07
311975,9319489,False,False,False,True,NaN,NaN,NaN,NaN
206454,9319489,False,False,False,True,53.000000,F,NaN,NaN
383668,9319489,False,False,False,True,56.764000,F,NaN,2015-08-07
122383,9319489,False,False,False,True,53.000000,F,NaN,NaN
352424,9319489,False,False,False,True,56.763859,F,NaN,2015-08-07
314841,9319489,False,False,False,True,56.763859,F,NaN,8/7/2015
363449,9319489,False,False,False,True,56.764000,F,NaN,8/7/2015
337184,9319489,False,False,False,True,56.763859,F,NaN,2015-08-07
